# Calculate Simplified Ikeda and insert into DB

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay.simplified_ikeda import calculate_roll_damping

import data
import copy
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Base, Model, LoadingCondition, Run, RolldecayLinear, RolldecayDirect, RolldecayNorwegian

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
sql="""
SELECT * from
rolldecay_direct_improved
INNER JOIN run
ON rolldecay_direct_improved.run_id == run.id
    INNER JOIN loading_conditions
    ON (run.loading_condition_id == loading_conditions.id)
        INNER JOIN models
        ON run.model_number == models.model_number
            INNER JOIN ships
            ON models.ship_name == ships.name

"""
df_rolldecay = pd.read_sql(sql, con=engine, index_col='run_id',)

In [ ]:
def calculate(row, PHI):
    
    LPP = row.lpp
    Beam = row.beam
    DRAFT = (row.TA + row.TF)/2
    
    lBK = row.BKL
    bBK = row.BKB
    OMEGA = row.omega0
    OG = row.kg
    CB = row.Volume/(row.lpp*row.beam*DRAFT)
    CMID = row.A0
    
    B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT = calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,lBK,bBK,OMEGA,DRAFT)
    s = pd.Series()
    s['B44HAT'] = B44HAT
    s['BFHAT'] = BFHAT
    s['BWHAT'] = BWHAT
    s['BEHAT'] = BEHAT
    s['BBKHAT'] = BBKHAT
    return s
    

In [ ]:
mask = ((df_rolldecay['lpp'] > 0) &
        (df_rolldecay['beam'] > 0) &
        (df_rolldecay['TA'] > 0) &
        (df_rolldecay['Volume'] > 0)
       )
df = df_rolldecay.loc[mask].copy()

In [ ]:
result = df.apply(func=calculate, PHI=10, axis=1)

In [ ]:
mask = result['B44HAT'] > 0
df = result.loc[mask]
df.hist('B44HAT', bins=50)

In [ ]:
mask = ((df['B44HAT'] > df['B44HAT'].quantile(0.00)) &
        (df['B44HAT'] < df['B44HAT'].quantile(0.98)) 
       )
df2 = df.loc[mask].copy()

In [ ]:
df2.hist('B44HAT', bins=20)

In [ ]:
sql="""
SELECT * from
rolldecay_linear
INNER JOIN run
ON rolldecay_linear.run_id == run.id
    INNER JOIN loading_conditions
    ON (run.loading_condition_id == loading_conditions.id)
        INNER JOIN models
        ON run.model_number == models.model_number
            INNER JOIN ships
            ON models.ship_name == ships.name

"""
df_rolldecay = pd.read_sql(sql, con=engine, index_col='run_id',)
df_rolldecay = df_rolldecay.loc[:,~df_rolldecay.columns.duplicated()]
#df_rolldecay = remove_outliers(df_rolldecay)
df_rolldecay.describe()

In [ ]:
df = df_rolldecay.dropna(subset=['Volume','beam'])

In [ ]:
mask = ((df_rolldecay['ship_speed']==0) & 
        (df_rolldecay[['Volume','beam']] > 0).all(axis=1)
       )
df = df.loc[mask].copy()

In [ ]:
df.describe()

In [ ]:
B44=df['zeta']*df['omega0']

B44.hist(bins=30)

In [ ]:
rho = 1000
g=9.81
Volume = df['Volume']
beam = df['beam']
B44_hat = B44/(rho*Volume*beam**2)*np.sqrt(beam/(2*g))

B44_hat.hist(bins=30)

In [ ]:
mask = ((B44_hat > B44_hat.quantile(0.00)) &
        (B44_hat < B44_hat.quantile(0.90)) 
       )
B44_hat2 = B44_hat[mask].copy()

B44_hat2.hist(bins=30)